In [45]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.datasets
import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder

# import mnist digits dataset
mnist = tf.keras.datasets.mnist # Object of the MNIST dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [28]:
# # DATA TRUNCATION FOR TEST
# x_train = x_train[:100]
# y_train = y_train[:100]
# x_test = x_test[:100]
# y_test = y_test[:100]

In [46]:
# Normalize the train dataset
x_train = tf.keras.utils.normalize(x_train, axis=1)
# Normalize the test dataset
x_test = tf.keras.utils.normalize(x_test, axis=1)

# Hilbert Curve translation

In [47]:
# Hilbert path generator
# ALL CREDIT TO: https://github.com/jakubcerveny/gilbert/blob/master/gilbert2d.py
# ALL CREDIT TO @jakubcerveny on GitHub

def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)

def generate_path(x,y,ax,ay,bx,by):
    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate_path(x, y, ax2, ay2, bx, by)
        yield from generate_path(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate_path(x, y, bx2, by2, ax2, ay2)
        yield from generate_path(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate_path(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))

def hilbert_path(n):
    yield from generate_path(0,0,n,0,0,n)


In [48]:
def dim_reduction(image):
    # Translate 2D image into 1D vector using Hilbert curve
    # image: 2D numpy array (28x28) or (nxn)
    # return: 1D numpy array
    width = image.shape[0]
    
    # Generate Hilbert curve path
    path = hilbert_path(width)
    # 1D vector widthxwidth
    vector = np.zeros(width*width)
    # Fill vector with image values
    for i, (x, y) in enumerate(path):
        vector[i] = image[x][y]
    return vector


In [32]:
# # Basic test
# print(np.array([[0,0,.25,.25],[0,0,.25,.25],[.75,.75,1,1],[.75,.75,1,1]]))
# print(dim_reduction(np.array([[0,0,.25,.25],[0,0,.25,.25],[.75,.75,1,1],[.75,.75,1,1]])))

### Translating MNIST

Note: This takes a long time!

In [49]:
x_train = np.array([dim_reduction(x) for x in x_train])
x_test = np.array([dim_reduction(x) for x in x_test])




In [50]:
print(x_train.shape)

y_train = np.array(y_train)
y_test = np.array(y_test)

# One-hot encode labels
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()

(60000, 784)


# Recurrent Neural Network

Problems w/ time series classification:
- LSTM RNN Just takes windows of data and does recognition based off of that
- Taking a sufficiently long window is literally just taking a snapshot of the image and running it thru RNN instead of CNN
- wtf bro lmao.

In [52]:
model = tf.keras.models.Sequential()
model.add(
    # keras.layers.Bidirectional(
      keras.layers.SimpleRNN(
          units=128,
          input_shape=[x_train.shape[1], 1]
      )
    # )
)
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dense(y_train.shape[1], activation='softmax'))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['acc']
)

In [53]:
history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/20
1688/1688 [==============================] - 230s 136ms/step - loss: 2.3386 - acc: 0.1032 - val_loss: 2.3018 - val_acc: 0.1050
Epoch 2/20
1688/1688 [==============================] - 222s 132ms/step - loss: 2.3022 - acc: 0.1123 - val_loss: 2.3018 - val_acc: 0.1050
Epoch 3/20
 449/1688 [======>.......................] - ETA: 2:39 - loss: 2.3014 - acc: 0.1118

KeyboardInterrupt: 